### Determining threshold
Testing all of the possible threshold values between 0 and 1.0 for hoop and mulch based on our training data

In [1]:
import pandas as pd
import numpy as np

In [43]:
# import datasets
# Santa Maria
sm_csv = 'N:/OCEANS_Program/Plastics/Agricultural_Plastics/AgPlastics_Pro/thresholding/SantaMaria_val_o_withPlasticProbs_p95.csv'
sm = pd.read_csv(sm_csv)

wv_csv = 'N:/OCEANS_Program/Plastics/Agricultural_Plastics/AgPlastics_Pro/thresholding/Watsonville_Points_BM_withPlasticProbs_p95_WY2022.csv'
wv = pd.read_csv(wv_csv)

ox_csv = 'N:/OCEANS_Program/Plastics/Agricultural_Plastics/AgPlastics_Pro/thresholding/Oxnard_Points_withPlasticProbs_p95_WY2022.csv'
ox = pd.read_csv(ox_csv)
# Convert 'blackmulch' to 'mulch' in the 'Type' column
ox['Type'] = ox['Type'].replace('blackmulch', 'mulch')

# print("Unique values in 'Type' column for Santa Maria dataset:", sm['Type'].unique())
# print("Unique values in 'Type' column for Watsonville dataset:", wv['Type'].unique())
# print("Unique values in 'Type' column for Oxnard dataset:", ox['Type'].unique())

# Define which dataset to use
data = ox

data = pd.concat([ox, wv, sm], ignore_index=True)


#### Comparing mulch and hoop confusion

If we take points that are classified as both mulch and hoop based on whatever thresholds we design, 
it will work most of the time. When a point is classified higher as mulch, it is mulch. 
There is some confusion:

- In Santa Maria, before thresholding, this approach would misclassify hoop as mulch 1.3% of the time, and misclassify mulch as hoop 7.4% of the time
- In Watsonville, before thresholding, this approach misclassifies hoop as mulch 0.6% of the time, and misclassifies mulch as hoop 0 times
- In Oxnard, no confusion
- Overall, 1% of hoop points would be misclassified as mulch and 2% of mulch points would be misclassified as hoop

In [45]:
hoop = data[data['Type'] == 'hoop']
hoop_count = hoop[hoop['hoop_p95'] > hoop['mulch_p95']].shape[0]
mulch_count = hoop[hoop['mulch_p95'] > hoop['hoop_p95']].shape[0]

if hoop_count == 0: # oxnard has no hoop points in 2022
    print('No points classified as "hoop"')
else:
    print('Of the points that are classified as "hoop"')
    print(f"Hoop count: {hoop_count}")
    print(f"Mulch count: {mulch_count}")
    print(f'Confusion rate: {mulch_count / (hoop_count + mulch_count)}')

mulch = data[data['Type'] == 'mulch']
hoop_count = mulch[mulch['hoop_p95'] > mulch['mulch_p95']].shape[0]
mulch_count = mulch[mulch['mulch_p95'] > mulch['hoop_p95']].shape[0]

print('Of the points that are classified as "mulch"')
print(f"Hoop count: {hoop_count}")
print(f"Mulch count: {mulch_count}")
print(f'Confusion rate: {hoop_count / (hoop_count + mulch_count)}')


Of the points that are classified as "hoop"
Hoop count: 296
Mulch count: 3
Confusion rate: 0.010033444816053512
Of the points that are classified as "mulch"
Hoop count: 11
Mulch count: 482
Confusion rate: 0.02231237322515213


In [47]:
# Function to calculate the accuracy for given thresholds based on our rules
def calculate_accuracy(thresholds, data):
    mulch_threshold, hoop_threshold = thresholds
    correct_classifications = 0
    
    for _, row in data.iterrows():
        if row['Type'] == 'mulch' and row['mulch_p95'] >= mulch_threshold:# and (row['hoop_p95'] < hoop_threshold or row['hoop_p95'] < row['mulch_p95']):
            correct_classifications += 1
        elif row['Type'] == 'hoop' and row['hoop_p95'] >= hoop_threshold:# and (row['mulch_p95'] < mulch_threshold or row['mulch_p95'] < row['hoop_p95']):
            correct_classifications += 1
        elif row['Type'] == 'other' and row['mulch_p95'] < mulch_threshold and row['hoop_p95'] < hoop_threshold:
            correct_classifications += 1
    
    return correct_classifications / len(data)

# Define the range of thresholds to test
thresholds_range = np.arange(0.0, 1.05, 0.05)

# Initialize variables to store the best thresholds and highest accuracy
best_thresholds = (0, 0)
highest_accuracy = 0

# Iterate over all possible combinations of thresholds
for mulch_threshold in thresholds_range:
    for hoop_threshold in thresholds_range:
        accuracy = calculate_accuracy((mulch_threshold, hoop_threshold), data)
        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            best_thresholds = (mulch_threshold, hoop_threshold)

print(f"Best thresholds: Mulch: {best_thresholds[0]}, Hoop: {best_thresholds[1]}")
print(f"Highest accuracy: {highest_accuracy}")

Best thresholds: Mulch: 0.45, Hoop: 0.7000000000000001
Highest accuracy: 0.8644747393744988


#### Resulting thresholds

**With the test for hoop/mulch confusion:**
- In Santa Maria, it's Mulch 0.5 and Hoop 0.75 (91% accurate)
- In Watsonville, it's Mulch 0.05 and Hoop 0.05 (81% accurate)
- In Oxnard, it's Mulch 0.45 and Hoop 0.35 (96% accurate)

Overall, it's Mulch 0.45 and Hoop 0.75 (85% accurate)

**Without the inter-plastic confusion test:**
- In Santa Maria, it's Mulch 0.5 and Hoop 0.5 (93% accurate)
- In Watsonville, it's the same - Mulch 0.05 and Hoop 0.05 (81% accurate)
- In Oxnard, it's the same -  Mulch 0.45 and Hoop 0.35 (96% accurate)

Overall, it's Mulch 0.45 and Hoop 0.70 (86% accurate)

Clearly something weird is going on in Watsonville with one of the classes
Could try again with 2021? 